# importar as bibliotecas

In [1]:
import spacy
import pandas
pandas.set_option('display.max_colwidth', None)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# executar as funções

In [59]:
def search_similar(phrase, comments) -> pandas.DataFrame:
	filterdData = []	
	similarities = []
	nlp = spacy.load("pt_core_news_sm")
	corpus_old = (list(comments['content'].values))
	corpus_old.insert(0, phrase) #concatena a phrase na base dados e coloca a phrase no inicio
    
	for i in corpus_old:
		filterdData.append(clear_phrase(i, nlp, ['PROPN', 'ADJ', 'NOUN', 'VERB'])) #fazendo uma limpeza nos comentários, deixando apenas Pronomes pessoais, adjetivos, substantivos e verbos 

	processedData = TfidfVectorizer().fit_transform(filterdData) #transformo a base de dados em vetores de números TF-IDF
	print(corpus_old[0])
	[lenght, M] = processedData.shape #pego o tamanho da base de dados    
	for i in range(1, lenght):
		s = cosine_similarity(processedData[0],processedData[i]) #o comentário processedData[0] é a frase digitada pelo usuário, daí preciso computar a distancia dessa frase com todos os comentários da base de dados
		similarities.append(s)
	comments['similaridade'] = similarities
    
	return comments
	
def clear_phrase(doc, nlp, pos_tag):
	if(type(doc) != spacy.tokens.doc.Doc):
		doc = nlp(doc)
	new_phrase = ' '.join([str(t.lemma_) for t in doc if t.pos_ in pos_tag])
	return new_phrase

def add_hours(minDate:str,maxDate:str)-> (str,str):
	minDate = minDate + "00:00:00"
	maxDate = maxDate + "23:59:59"

	return minDate,maxDate
def get_comments(originPath:str,minDate:str = None,maxDate:str = None,versions = None) -> pandas.DataFrame:
	
	commentsFile = pandas.read_csv(originPath)

	if minDate != None and maxDate != None:
		minDate,maxDate = add_hours(minDate,maxDate)

	for i in range(0, len(commentsFile)):
		if commentsFile['date'][i] < minDate or commentsFile['date'][i] > maxDate or commentsFile['version'][i] not in versions:
			#print("droped",commentsFile['version'][i])
			commentsFile = commentsFile.drop([i])
	return commentsFile

def sort_by_thumbs(file:pandas.DataFrame) -> pandas.DataFrame:
	file = file.sort_values(by=['thumbs'])
	return file

def limpar_comentarios(df, pos_tag):
    nlp = spacy.load("pt_core_news_sm")
    lista = []
    for i in df['content']:
        r = clear_phrase(i, nlp, pos_tag) #deixando os comentários apenas com pos_tag
        lista.append(r) 
    df['comentarios_limpos'] = lista
    return df

def extrair_ngrams(corpus, AA, BB):
    c_vec = CountVectorizer(ngram_range=(AA,BB))
    ngrams = c_vec.fit_transform(corpus)
    count_values = ngrams.toarray().sum(axis=0)
    vocab = c_vec.vocabulary_
    df_ngram = pandas.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'ngram'})
    
    return df_ngram

def plotar_frequencias(variavel, df):
    copy_df = df[[variavel]].copy()
    copy_df.groupby(new[variavel].dt.date).count().plot(kind="bar")
    
def analisar_verbos(verbo, nota, df):
    similarities_list2 = search_similar(verbo, df[df['score']==nota])
    d5 = similarities_list2.sort_values('similaridade', ascending=False)
    d6 = d5.sort_values('thumbs', ascending=False)
    return d6
    



# coletar os comentários

In [3]:
database = get_comments("./General_Data.csv","2021-01-01","2021-10-01","3.0.0")

# identificar tópicos mais comentados

In [60]:
A = limpar_comentarios(database, ['VERB', 'NOUN'])

In [66]:
B = extrair_ngrams(A['comentarios_limpos'], 3, 3)
B[B['frequency']>10]

,frequency,ngram
0,26,conseguir fazer reconhecimento
1,15,conseguir recuperar senha
2,11,opção criar contar
3,11,fazer provar vidar


# buscar por comentários similares

In [68]:
similarities_list = search_similar('facial', database)
d = similarities_list.sort_values('similaridade', ascending=False)

facial


In [67]:
d2 = d[d['similaridade'] >= 0.1] #exibindo apenas os comentários com similaridade >= 0.3
d2

,Unnamed: 0,date,content,score,version,thumbs,similaridade
673,827,2021-09-23 19:01:42,reconhecimento facial não era,1,3.0.0,0,[[0.6953729889424185]]
453,559,2021-09-28 16:50:57,Pra quê reconhecimento facial.... 😤🤦🏻,1,3.0.0,0,[[0.6953729889424185]]
18917,1155,2021-09-27 21:24:26,ok. Reconhecimento facial!,5,3.0.0,0,[[0.6953729889424185]]
10890,140,2021-09-25 21:04:40,não funciona o reconhecimento facial,3,3.0.0,0,[[0.5301660582904462]]
404,503,2021-09-29 13:20:36,Reconhecimento facial não funciona,1,3.0.0,0,[[0.5301660582904462]]
...,...,...,...,...,...,...,...
446,552,2021-09-28 17:47:38,"Impossível para uma pessoa com 70 anos uso obrigatório de óculos fazer o cadastramento facial. O tempo é escasso e vai aparecendo a mensagem tempo esgotado tente novamente, 1,2,3,4 ........n.x Poderiam utilizar as digitais do banco, foto da CNH, IR. Carteira de identidade, passaporte, enfim algo que funcione.",1,3.0.0,0,[[0.10469520865025228]]
543,663,2021-09-27 09:58:47,"Pouco tempo para fazer o reconhecimento facial, meu pai idoso não consegue. E mesmo quando consegue, vem a mensagem "" ocorreu um erro inesperado"". Se tenta cadastro por meio do banco, dá erro. Fazem app nas pressas e fica isso aí. Da impressão de segurança mas vazam os dados de todo cidadão na internet.",1,3.0.0,2,[[0.10411372366257399]]
619,758,2021-09-24 18:05:35,"Bom, estou tentando entrar, porém não consigo. Pois nunca me cadastrei em nada e quando dígito meu CPF diz pra mim colocar minha senha atual, mas não tenho nenhuma senha. Então tento criar uma nova senha, com reconhecimento facial e diz que não tenho nenhum registro Biométrico no sistema. ESTOU PERDIDO, POR FAVOR ME AJUDEM!",1,3.0.0,0,[[0.10269450899145406]]
966,1180,2021-09-17 13:16:17,"Pena que uma estrela é o minimo, porque no meu ponto de vista não condiz com a minha insatisfação! Instalei o App e já tentei mais de uma vez me cadastrar limpei os dados e o cache, faço o reconhecimento facial e toda vez aparece a seguinte mensagem: ""Internal Server Error"".",1,3.0.0,19,[[0.10221711260638756]]


In [ ]:
d2["date"] = d2["date"].astype("datetime64")
d2.groupby(d2['date'].dt.date).count().plot(kind="bar", legend=None)

In [70]:
d3 = limpar_comentarios(d2, ['VERB']) #deixando apenas os verbos nos comentários

In [71]:
d4 = d3[d3['score']==5]
n = extrair_ngrams(d4['comentarios_limpos'], 1, 1)
n[n['frequency']>1] #exibindo os verbos mais utilizados pelos comentários com nota = 5

,frequency,ngram
0,8,fazer
1,7,conseguir
2,3,tentar
3,3,gostar
4,2,poder
5,2,melhorar
6,2,dever
7,2,achar


In [78]:
analisar_verbos('funciona', 3, d2) #mostrando os comentários com mais curtidas com relacionado ao verbo

funciona


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,date,content,score,version,thumbs,similaridade,comentarios_limpos
12101,1399,2021-04-25 22:35:21,Reconhecimento facial não funciona. Movimentos para direita ou esquerda Nada dá certo. Porque fazer o reconhecimento facial toda vez que acesso o app?,3,3.0.0,86,[[0.21445454498819674]],funcionar dar fazer
12057,1355,2021-04-29 01:05:19,"Bem difícil de usar. Reconhecimento facial só funcionou hoje à noite, horas depois de ter instalado o app e quase ter desistido.",3,3.0.0,46,[[0.23532077281099847]],usar funcionar instalar desistir
10850,96,2021-09-28 20:26:46,Olha vai me descupar mais nao foi ate hoje e pior fala que foi cancelado a leitura do rosto digital facial,3,3.0.0,6,[[0.0]],descupar ser falir cancelar
12028,1324,2021-05-01 14:39:33,"Muito frustante pois a validação facial não funciona de tanto tentar já travou o app, tentei mais vezes em outros dias e assim mesmo não funciona!",3,3.0.0,2,[[0.4206731258705905]],funcionar tentar travar tentar funcionar
10854,101,2021-09-28 18:19:17,"Falta mais informações como, quem passou a ser pensionista a 6 meses dentro do ano de 2021, precisa fazer a prova de vida, porém fazer o reconhecimento facial e praticamente impossível , porém se estou devidamente identificada dentro da minha situação poderia dizer se preciso ou não fazer a prova de vida, pois estou com prado Vencendo 30/09/21 e ainda não consegui nada.",3,3.0.0,1,[[0.0]],passar pensionista preciso fazer fazer identificar poder dizer fazer Vencendo conseguir
10940,191,2021-09-22 15:09:15,"avaliação facial deprimente, espero melhorias para a próxima vez que eu necessitar de prova de vida. ""fora isso, tuuuudo bem!"" Em tempo: porquê só pede a CNH e não o título de eleitor? E quem não tem CNH?",3,3.0.0,1,[[0.0]],esperar necessitar pedir ter
10931,182,2021-09-22 19:32:04,"reconhecimento facial complicado, tive que fazer diversas tentativas.",3,3.0.0,1,[[0.0]],ter fazer
10898,148,2021-09-25 00:00:00,"muito chata a parta do reconhecimento facial, quase que não consegui",3,3.0.0,0,[[0.0]],conseguir
10962,213,2021-09-21 00:50:53,reconhecimento facial não me reconhece,3,3.0.0,0,[[0.0]],reconhecer
10974,226,2021-09-19 19:02:35,muita dificuldade com o reconhecimento facial,3,3.0.0,0,[[0.0]],
